# Intune Module Setup Example Notebook

This notebook creates 1 table (devices) into a new Spark database called s2p_intune_module. 

### Provision storage accounts

The storage account variable has to be changed to the name of the storage account associated with your Azure resource group.

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, TimestampType, DateType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


# data lake and container information
storage_account = 'stoeadaihackathon'
use_test_env = False

if use_test_env:
    stage1np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1np'
    stage2np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2np'
    stage2p = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2p'
else:
    stage1np = 'abfss://stage1np@' + storage_account + '.dfs.core.windows.net'
    stage2np = 'abfss://stage2np@' + storage_account + '.dfs.core.windows.net'
    stage2p = 'abfss://stage2p@' + storage_account + '.dfs.core.windows.net'

StatementMeta(medium1, 11, 2, Finished, Available)

### Load Raw Data from Lake
To ensure that that the right tables are loaded, confirm that the file paths match your data lake storage containers. 

The top code-block defines the schema of how the devices CSV file in stage 1 is stored.

In [13]:
# load needed device table from the CSV in the data lake storage
dfIntuneDevices = spark.read.format('csv').load(f'{stage1np}/intune_module/devices/*.csv', header='true')

StatementMeta(medium1, 11, 3, Finished, Available)

## 1. Devices table
Contains all devices (from students and teachers) at a school-system level

** Databases and tables used: **

 - None

**CSV files used:**

- the file from: intune_module/devices/*.csv

**Database and table created:**

1. Spark DB: s2p_intune_module
- Table: devices

In [14]:
display(dfIntuneDevices.limit(10))

StatementMeta(medium1, 11, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 13e1c7fd-9eea-4fd1-9447-54bf8f1853dc)

### Add an additional column "accessOutsideOfSchool"

This "accessOutsideOfSchool" column uses the "lastCheckIn" column information to determine if a specific student's device has access outside of school, based on the conditions:

 - If the last check in was on a weekend (i.e. Saturday or Sunday), then "accessOutsideOfSchool" is true.
 - If the last check in was before 9 AM (9:00) on a weekday, then "accessOutsideOfSchool" is true.
 - If the last check in was after 4 PM (16:00) on a weekday, then "accessOutsideOfSchool" is true. 

Otherwise, "accessOutsideOfSchool" is defaulted to false. 

Also this code creates a "lastCheckInDate" date-type column, used for the date filter in the PowerBI dashboard.

In [18]:
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInTime', split(col('lastCheckIn'), ' ').getItem(1))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDate', split(col('lastCheckIn'), ' ').getItem(0))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInHourOfDay', split(col('lastCheckInTime'), ':').getItem(0))
dfIntuneDevices = dfIntuneDevices.drop('lastCheckInTime')

dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDate', col('lastCheckInDate').cast("date"))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDayOfWeek', date_format(col('lastCheckIn'), "E"))

dfIntuneDevices = dfIntuneDevices.withColumn('AccessOutsideOfSchool', when(col('lastCheckInDayOfWeek') == "Sat", "true").otherwise(when(col('lastCheckInDayOfWeek') == "Sun", "true").otherwise(when(col('lastCheckInHourOfDay') >= 16, "true").otherwise(when(col('lastCheckInHourOfDay') < 9, "true").otherwise("false")))))
# Can comment out this drop if you don't want to drop these two columns
dfIntuneDevices = dfIntuneDevices.drop('lastCheckInDayOfWeek').drop('lastCheckInHourOfDay')

display(dfIntuneDevices.limit(10))

StatementMeta(medium1, 11, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 87cfeedb-db14-4da8-9215-709d4560fc0d)

## Write Data Back to Lake

### Writing to Stage 2np

In [20]:
# write back to the lake in stage 2 ds2_main directory
dfIntuneDevices.coalesce(1).write.format('parquet').mode('overwrite').save(stage2np + '/intune_module/devices')

StatementMeta(medium1, 11, 10, Finished, Available)

### Writing to Stage 2p
Pseudonymizing the primaryUserUPNs (userPrincipalNames) data from the devices CSV.

In [21]:
%run /OEA_py

StatementMeta(, 11, -1, Finished, Available)

In [22]:
oea = OEA()

StatementMeta(medium1, 11, 12, Finished, Available)

2021-10-20 16:12:14,790 - OEA - DEBUG - OEA initialized.
OEA initialized.

In [23]:
devicesSchema = [['DeviceName', 'string', 'no-op'],
                            ['ManagedBy', 'string', 'no-op'],
                            ['Ownership', 'string', 'no-op'],
                            ['Compliance','string','no-op'],
                            ['OS', 'string', 'no-op'],
                            ['OSVersion', 'string', 'no-op'],
                            ['LastCheckIn', 'timestamp', 'no-op'],
                            ['PrimaryUserUPN', 'string', 'hash'],
                            ['DeviceID', 'string', 'no-op'],
                            ['LastCheckInDate', 'date', 'no-op'],
                            ['AccessOutsideOfSchool', 'boolean', 'no-op']]

df_pseudo, df_lookup = oea.pseudonymize(dfIntuneDevices, devicesSchema)

df_pseudo.coalesce(1).write.format('parquet').mode('overwrite').save(stage2p + '/intune_module/devices')

StatementMeta(medium1, 11, 13, Finished, Available)

### Load to Spark DB

In [24]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"DROP TABLE IF EXISTS {db_name}.devices")
    spark.sql(f"create table if not exists {db_name}.devices using PARQUET location '{source_path}'")
    
create_spark_db('s2p_intune_module', stage2p + '/intune_module/devices')

StatementMeta(medium1, 11, 14, Finished, Available)

## Reset Data Processing

In [ ]:
# Uncomment line 7 if you would like to walk through the process again from the beginning
def reset_all_processing():
    oea.rm_if_exists(stage2np + '/intune_module/devices')
    oea.rm_if_exists(stage2p + '/intune_module/devices')
    oea.drop_db('s2p_intune_module')

#reset_all_processing()